I am using spark-2.4.0-bin-hadoop2.7 along with jdk 8 and Kafka 2.12-2.1.1 .
Note: I got issues with jdk 11 while using the RDD object, so, downgraded to jdk8.

Pre-requisite - 
1) download http://central.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.0/spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar tp SPARK_HOME\jars  directory. 
2) add this jar to path using following command before getting the stream object as given below in the code - 
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages C:\\myprograms\\spark-2.4.0-bin-hadoop2.7\\jars\\spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar pyspark-shell' #note that the "pyspark-shell" part is very important!!.



In [3]:
#import findspark

import pyspark
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
#udf, concat, desc, window
from pyspark.streaming.kafka import KafkaUtils

from pyspark.sql.types import *
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
# Use tqdm to show progress of an pandas function we use
tqdm.pandas()


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Create SparkSql Session¶
local[*]  - use local node with as many worker threads as logical cores on your machine.



In [4]:
spark=SparkSession.builder.appName(name="kafka_stream").master("local[*]").getOrCreate()


# Create streaming context with window=10secs

In [5]:
ssc=StreamingContext(spark.sparkContext,10)

In [6]:
zookeeper="localhost:2181"#:9092"
groupid="sparkconsumer"
topic_name="topic_events"
partitions=3
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages C:\\myprograms\\spark-2.4.0-bin-hadoop2.7\\jars\\spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar pyspark-shell' #note that the "pyspark-shell" part is very important!!.

stream = KafkaUtils.createStream(ssc, zookeeper, groupid, {topic_name:partitions})

# Extract the events from the Dstream object.

In [7]:
#stream has data like this - (None, 'd1 e1 f1')
#stream.pprint()

dstream = stream.map(lambda x: x[1])
#lines contains the content(values) from kafka stream (k,v) pair.

#lines.pprint()
#print(type(lines))


# Group the events that come within 20 seconds window, with sliding window=10 secs.
Result dstream of this function will contain a single RDD.
This RDD will contain all event messages we received within that sliding window of 20 mins.

In [8]:
dstream=dstream.reduceByWindow(lambda x,y: "{} {}".format(x,y),
                     None,20,10)

In [9]:
#this is a sample function.
#In-parameter contains the list of events we received during the sliding window of 20 seconds.
#This one just prints the list. However, ideally, we can send this list over to a processing thread.
def process_event_groups(lst):
    print(lst)

# Pass on the event list within the window to a processing function.
The reduced dstream contains single RDD.
<br/> Pass on the contents of this RDD to the processing function.
<br/> ie., The list of events within the 20 mins sliding window will be passed on to the processing function.

In [10]:
dstream.foreachRDD(lambda x: process_event_groups(x.collect()))

# Start receiving from the stream. This will execute all the functions on the 'stream' object that we defined above, for each of the '10' seconds window.
As we are using the reduceByWindow, the events will be grouped as per this function.

In [ ]:
ssc.start()
ssc.awaitTermination()

[]
[]
['a1 b1 c1 d1 e1']
['a1 b1 c1 d1 e1 f1 g1']
['f1 g1 h1 j1']
['h1 j1 k1 l1 m1']
['k1 l1 m1']
['n1 o1 p1']
['n1 o1 p1']
